In [ ]:
%matplotlib inline

source : https://nilearn.github.io/auto_examples/plot_decoding_tutorial.html#sphx-glr-download-auto-examples-plot-decoding-tutorial-py


# A introduction tutorial to fMRI decoding

Here is a simple tutorial on decoding with nilearn. It reproduces the
Haxby 2001 study on a face vs cat discrimination task in a mask of the
ventral stream.

    * J.V. Haxby et al. "Distributed and Overlapping Representations of Faces
      and Objects in Ventral Temporal Cortex", Science vol 293 (2001), p
      2425.-2430.

This tutorial is meant as an introduction to the various steps of a decoding
analysis using Nilearn meta-estimator: :class:`nilearn.decoding.Decoder`

## Retrieve and load the fMRI data from the Haxby study

### First download the data

The :func:`nilearn.datasets.fetch_haxby` function will download the
Haxby dataset if not present on the disk, in the nilearn data directory.
It can take a while to download about 310 Mo of data from the Internet.



In [ ]:
from nilearn import datasets
# By default 2nd subject will be fetched
haxby_dataset = datasets.fetch_haxby()
# 'func' is a list of filenames: one for each subject
fmri_filename = haxby_dataset.func[0]

# print basic information on the dataset
print(f'First subject functional nifti images (4D) are at: {fmri_filename}')  # 4D data

### Visualizing the fmri volume

In [ ]:
from nilearn import plotting
from nilearn.image import mean_img

# Only 3D images can be plotted, not 4D so we average over time with mean_img
plotting.view_img(mean_img(fmri_filename), threshold=none)

### Feature extraction: from fMRI volumes to a data matrix

These are some really lovely images, but for machine learning
we need matrices to work with the actual data. Fortunately, the
:class:`nilearn.decoding.Decoder` object we will use later on can
automatically transform Nifti images into matrices.
All we have to do for now is define a mask filename.

A mask of the Ventral Temporal (VT) cortex coming from the
Haxby study is available:



In [ ]:
mask_filename = haxby_dataset.mask_vt

# Let's visualize it, using the subject's anatomical image as a background
plotting.plot_roi(mask_filename, bg_img=haxby_dataset.anat, cmap='paired')

In [ ]:
mask_filename

### Load the behavioral labels

In [ ]:
import pandas as pd

behavioral = pd.read_csv(haxby_dataset.session_target[0], delimiter=',')
print(behavioral)

The task was a visual-recognition task, and the labels denote the
experimental condition: the type of object that was presented to the
subject. This is what we are going to try to predict.



In [ ]:
conditions = behavioral['labels']
print(conditions.unique())

### Restrict the analysis to cats and faces

As we can see from the targets above, the experiment contains many
conditions. As a consequence, the data is quite big. Not all of this data
has an interest to us for decoding, so we will keep only :term:`fmri<fMRI>` signals
corresponding to faces or cats. We create a mask of the samples belonging to
the condition; this mask is then applied to the :term:`fmri<fMRI>` data to restrict the
classification to the face vs cat discrimination.

The input data will become much smaller (i.e. :term:`fmri<fMRI>` signal is shorter):



In [ ]:
condition_mask = conditions.isin('face', 'cat')

Because the data is in one single large 4D image, we need to use
index_img to do the split easily.



In [ ]:
from nilearn.image import index_img
fmri_niimgs = index_img(fmri_filename, condition_mask)

We apply the same mask to the targets



In [ ]:
conditions = conditions[condition_mask].values

## Decoding with Support Vector Machine

As a decoder, we use a Support Vector Classifier with a linear kernel. We
first create it using by using :class:`nilearn.decoding.Decoder`.

Let's leave out the 30 last data points during training, and test the
prediction on these 30 last points:

In [ ]:
from nilearn.decoding import Decoder

fmri_niimgs_train = index_img(fmri_niimgs, slice(0, -30))
fmri_niimgs_test = index_img(fmri_niimgs, slice(-30, None))
conditions_train = conditions[:-30]
conditions_test = conditions[-30:]

decoder = Decoder(estimator='SVC', mask=mask_filename, standardize=True)
decoder.fit(conditions_train, fmri_niimgs_train)

prediction = decoder.predict(fmri_niimgs_test)
print(prediction)

In [ ]:
test_accuracy = (prediction == conditions_test).sum() / len(conditions_test)
print(f"Prediction Accuracy: {test_accuracy}")

## Inspecting the model weights

Finally, it may be useful to inspect and display the model weights.

### Turning the weights into a nifti image

We retrieve the SVC discriminating weights



In [ ]:
coefficients = decoder.coef
print(coefficients)

It's a numpy array with only one coefficient per voxel.



To get the Nifti image of these coefficients, we only need retrieve the
`coef_img_` in the decoder and select the class



In [ ]:
coef_img = decoder.coef_img_['Face']

### Plotting the SVM weights

We can plot the weights, using the subject's anatomical as a background



In [ ]:
plotting.view_img(
    decoder.coef_img_['face'], bg_img=haxby_dataset.anat[0],
    title="SVM weights", dim=-1
)